In [ ]:
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
image_dir='Z:/ortofotomapa/images/'
tiffs = glob(image_dir+'/*.tif')
len(tiffs)

In [ ]:
fname = tiffs[0]
print(fname)

In [ ]:
img = cv2.imread(fname)
img.shape

In [ ]:
plt.imshow(img[:,:,::-1])

# Rasterio

In [ ]:
import rasterio
from rasterio.plot import show

In [ ]:
raster = rasterio.open(fname)
show(raster)

In [ ]:
dir(raster)

In [ ]:
print(img.width, img.height)
print(img.crs)
print(img.transform)
print(img.count)
print(img.indexes)
print(img.bounds)

In [ ]:
print(img.transform)


In [ ]:
img.transform * (0,0), img.transform * (img.width, img.height)

# EOMaps?

In [ ]:
# from eomaps import Maps


# m = Maps(crs=2180, figsize=(10, 10))
# m.add_wms.OpenStreetMap.add_layer.stamen_terrain()
# # plot shapefile (zorder=2 to be on top of the DEM)
# # m.add_gdf(shp_data, column=shp_data.VEGKOD, cmap="viridis", ec="k", lw=0.2, alpha=0.5, zorder=2)
# # plot DEM
# m2 = m.new_layer_from_file.GeoTIFF(tiffs[0], zorder=1)

# m.ax.set_extent((589913.0408156103, 713614.6619114348, 7495264.310799116, 7618965.93189494),
#                 Maps.CRS.epsg(2180))

In [ ]:
import geopandas as gpd

In [ ]:

shp_data = gpd.read_file("C:/Users/karol/projects/cufix/bdot10kseg/notebooks/data/unzipped/SHP/PL.PZGiK.333.0801__OT_BUBD_A.shp")
shp_data

In [ ]:
plt.figure(figsize=(20,10))
shp_data.X_KOD.hist()

In [ ]:
shp_data.X_KOD.value_counts()

In [ ]:
shp_data.geometry.plot()

In [ ]:
shp_data['geometry1992'] = shp_data.geometry.to_epsg(2180)

In [ ]:
shp_data.geometry1992.plot()

In [ ]:
shp_data.crs

In [ ]:
shp_data.geometry.iloc[0]

In [ ]:
shp_data.geometry.iloc[0].exterior

In [ ]:
shp_data.geometry.iloc[0].exterior.xy

# Arkusze

In [ ]:
godlo1992 = gpd.read_file(r"C:\Users\karol\projects\cufix\bdot10kseg\data\SiatkaPodzialuArkuszowegoNMT_PL1992_5000\PL1992_5000.shp")

godlo1992

In [ ]:
godlo1992.geometry.plot(figsize=(10,10))

In [ ]:
downloaded_godlos = []
for t in tiffs:
    downloaded_godlos.append(t.split('/')[-1].split('_')[-1][:-4])

In [ ]:
downloaded_godlos[:15]

In [ ]:
'M-34-9-B-c-4-1' in godlo1992.godlo

In [ ]:
godlo1992['downloaded'] = [_ in downloaded_godlos for _ in godlo1992.godlo]
godlo1992

In [ ]:
godlo1992.downloaded.value_counts()